In [ ]:
import time
from random import random
from selenium import webdriver
from logging import raiseExceptions
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
service = Service()
options = webdriver.ChromeOptions()
options.add_extension("adblock.crx")
options.add_argument("--disable-application-cache")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-setuid-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.page_load_strategy = 'eager'
prefs = {
    "profile.managed_default_content_settings.images": 2,
    "profile.managed_default_content_settings.stylesheets": 2,
    "profile.managed_default_content_settings.plugins": 2,
    "profile.managed_default_content_settings.popups": 2,
    "profile.managed_default_content_settings.geolocation": 2,
    "profile.managed_default_content_settings.notifications": 2,
    "profile.managed_default_content_settings.media_stream": 2,
    "profile.managed_default_content_settings.javascript": 1,
    "profile.managed_default_content_settings.frame": 2
}

options.add_experimental_option("prefs", prefs)
options.add_experimental_option("prefs", prefs)
options.set_capability("pageLoadStrategy", "eager")

In [ ]:
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
driver.get('https://lupa.uol.com.br/busca/elei%C3%A7%C3%B5es')
tag = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located(
        (By.XPATH,
            '//*[@id="root"]/div[2]/div[1]/div/div/div/div[2]/div/div[2]/div[1]/div[1]'
         )
    )
)
tag.click()
(driver.find_element(By.XPATH,
 '//*[@id="root"]/div[2]/div[1]/div/div/div/div[2]/div/div[2]/div[1]/div[2]/div/div[1]/span')).click()
driver.execute_script("window.scrollBy(0, 200);")
(driver.find_element(By.XPATH,
 '//*[@id="root"]/div[2]/div[1]/div/div/div/div[2]/div/div[2]/div[1]/div[2]/div/div[11]/button')).click()

In [ ]:
num = ''
while (num != 'todos'):
    driver.execute_script("window.scrollBy(0, 3000);")
    time.sleep(1)
    num = driver.find_element(
        By.XPATH, '//*[@id="init"]/div/div/div[1]/div/span').text.split()[1]

In [ ]:
list = ['#', '\'', '/', ',', '.', '', ']', '[', '(', ')', '*', '@', '!', '?', '|', ':', ';', '&', '–', '”', '“', '…', '=', ',', '.', '–', '"', '‘', '’']
trainNews = []
testNews = []
aux = []

try:
    newsCards = WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located(
            (By.XPATH,
                '//span[@class="sc-eDvSVe hAIjQn"]'
                )
        )
    )
except:
    raise Exception('elemento não encontrado')

for card in newsCards:
    #  Scroll to the card and click
    driver.execute_script("arguments[0].scrollIntoView({behavior:'smooth',block:'center'});", card)
    link = card.find_element(By.XPATH, ".//ancestor::a[1]").get_attribute("href")
    driver.execute_script(f"window.open('{link}', '_blank');")

    # Change the driver window
    driver.switch_to.window(driver.window_handles[-1])

    # Search all the news and store them
    try:
        WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='root']"))
        )

        newsArray = []
        tagsArray = []

        newsElements = WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.XPATH,
            '//span/b'
        ))
        )
        for news in newsElements:
            if news.text.strip() != "Lupa":
                newsArray.append(news.text)


        tag = WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.XPATH,
            '//span[@class="sc-eDvSVe bMMjGB"]'
        ))
        )

        for i in tag:
            tagsArray.append(i.text)

        tagsArray.pop(0)
        
        for news, tag in zip(newsArray, tagsArray):
            if tag == "Falso":
                aux = []
                if(random() < 0.8):
                    text = news.strip().split()
                    for words in text:
                        for char in list:
                            words = words.replace(char, '')
                        aux.append(words.lower())

                    aux = [word for word in aux if word != '']
                    aux.insert(0, 'f')
                    trainNews.append(aux)
                else:
                    text = news.strip().split()
                    for words in text:
                        for char in list:
                            words = words.replace(char, '')
                        aux.append(words.lower())

                    aux = [word for word in aux if word != '']
                    aux.insert(0, 'f')
                    testNews.append(aux)

    except Exception as e:
        print("error:", e)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        continue

    # Close the window
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

In [ ]:
arqOut = open('./TRAINfakenewsLupa.txt', 'w', encoding='utf-8')
for news in trainNews:
    print(news, file=arqOut)
arqOut.close()

In [ ]:
arqOut = open('./TESTfakenewsLupa.txt', 'w', encoding='utf-8')
for news in testNews:
    print(news, file=arqOut)
arqOut.close()